In [ ]:
# Install necessary libraries
!pip install nltk tensorflow scikit-learn numpy streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import os
import nltk
import ssl
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [20]:
# Download necessary NLTK data
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))

In [21]:
# Define intents
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up"],
        "responses": ["Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    }
]

In [22]:
# Prepare data for LSTM
patterns = []
tags = []
responses = []
for intent in intents:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
        responses.append(intent['responses'])

In [23]:
# Encode the tags (categories)
label_encoder = LabelEncoder()
tags_encoded = label_encoder.fit_transform(tags)

In [24]:
# Tokenize the patterns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)
X = tokenizer.texts_to_sequences(patterns)

In [25]:
# Pad sequences to make them of equal length
X = pad_sequences(X, padding='post')

In [26]:
# Train-Test Split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, tags_encoded, test_size=0.2, random_state=42)

In [27]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=X.shape[1]))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=len(set(tags)), activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [28]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [29]:
# Train the model
model.fit(X_train, np.array(y_train), epochs=200, batch_size=8, validation_data=(X_test, np.array(y_test)))

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - accuracy: 0.0000e+00 - loss: 2.2052 - val_accuracy: 0.1429 - val_loss: 2.1936
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1458 - loss: 2.1867 - val_accuracy: 0.0000e+00 - val_loss: 2.1918
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2448 - loss: 2.1744 - val_accuracy: 0.0000e+00 - val_loss: 2.1915
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2031 - loss: 2.1595 - val_accuracy: 0.0000e+00 - val_loss: 2.1891
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2292 - loss: 2.1386 - val_accuracy: 0.0000e+00 - val_loss: 2.1860
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2604 - loss: 2.1015 - val_accuracy: 0.0000e+00 - val_loss: 2.1839
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2656 - loss: 2.0811 - val_accuracy: 0.0000e+00 - val_loss: 2.1852
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2344 - loss: 2.0342 - val

In [30]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, np.array(y_test))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4286 - loss: 3.2668
Test Accuracy: 42.86%


the model achieved a test accuracy of 42.86%, which is fairly low this is not unusual when working with small datasets, simple models, or unoptimized configurations

In [31]:
# Function to predict the response using LSTM model
def chatbot(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=X.shape[1], padding='post')
    pred_tag = model.predict(input_seq)
    tag = label_encoder.inverse_transform([np.argmax(pred_tag)])

    # Find the corresponding response
    for intent in intents:
        if intent['tag'] == tag[0]:
            response = random.choice(intent['responses'])
            return response

In [32]:
# Test the chatbot
test_input = "Hello"
response = chatbot(test_input)
print("Chatbot response:", response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Chatbot response: Nothing much


In [33]:
# Install ngrok to create a public URL for Streamlit in Colab
!pip install pyngrok

# Create a script to run Streamlit (save it as 'app.py')
%%writefile app.py
import streamlit as st
import random
import numpy as np

# Your chatbot code here

# Streamlit UI and chatbot code...

# Run the Streamlit app
!streamlit run app.py & npx ngrok http 8501

UsageError: Line magic function `%%writefile` not found.


In [34]:
test_input = "Goodbye"
response = chatbot(test_input)
print("Chatbot response:", response)

test_input = "Can you help me?"
response = chatbot(test_input)
print("Chatbot response:", response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Chatbot response: See you later
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Chatbot response: I'm here to help. What's the problem?
